In [ ]:
# import os
# import shutil
# from PIL import Image
# from paddleocr import PaddleOCR
# from tqdm import tqdm
# from datetime import datetime
# import math
# import time
# import numpy as np
# import pandas as pd

# # 定义基础路径
# base_path = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'

# # 初始化 PaddleOCR
# ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

# def move_images_with_white_pixels(source_folder, white_folder, threshold=0.40):
#     image_files = [f for f in os.listdir(source_folder) if f.endswith(('.jpg', '.png'))]
    
#     with tqdm(total=len(image_files), desc=f"Moving images from {source_folder}") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(source_folder, filename)
            
#             with Image.open(img_path) as img:
#                 if img.mode != 'RGB':
#                     img = img.convert('RGB')
#                 width, height = img.size
            
#                 white_pixels = sum(1 for x in range(width) for y in range(height) 
#                                    if all(v > 230 for v in img.getpixel((x, y))[:3]))
                
#                 white_ratio = white_pixels / (width * height)
                
#                 if white_ratio > threshold:
#                     target_path = os.path.join(white_folder, filename)
#                     try:
#                         shutil.move(img_path, white_folder)
#                         pbar.write(f"Image '{filename}' moved to white folder.")
#                     except Exception as e:
#                         pbar.write(f"Error moving image '{filename}': {e}")
                        
#             pbar.update(1)

# def process_subfolder(subfolder_path):
#     grounding_output_path = os.path.join(subfolder_path, 'grounding_output')
#     if not os.path.exists(grounding_output_path):
#         # print(f"No grounding_output folder in {subfolder_path}")
#         return

#     # 创建分类文件夹
#     price_folder = os.path.join(grounding_output_path, 'price')
#     txt_folder = os.path.join(grounding_output_path, 'txt')
#     scene_folder = os.path.join(grounding_output_path, 'scene')
#     white_folder = os.path.join(grounding_output_path, 'white')

#     for folder in [price_folder, txt_folder, scene_folder, white_folder]:
#         if not os.path.exists(folder):
#             os.makedirs(folder)

#     # 获取所有图片文件
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

#     # Step 1: 筛选价促卖点图
#     print('Step 1: 筛选价促卖点图')
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(grounding_output_path, filename)
            
#             result = ocr.ocr(img_path, cls=True)
            
#             if not result:
#                 pbar.update(1)
#                 continue
            
#             data_list = result[0]
            
#             keywords = ['满', '减', '折', '到手价', '送', '免息', '活动价', '包邮价', '参考价', 
#                         r'.*满.*减.*', r'.*满.*-.*', r'.*满.*赠.*',r'.*满.*送.*', r'.*价.*', 
#                         '券', '优惠', '用券', '领券', '券', '送', '低至', '立减', '直降', '免息',
#                         '¥', '夫', '￥','分钱']
            
#             contains_keyword = any(any(keyword in data[1][0] for keyword in keywords) for data in data_list)
            
#             if contains_keyword:
#                 try:
#                     shutil.move(img_path, price_folder)
#                     pbar.write(f"Image '{filename}' moved to price_folder.")
#                 except Exception as e:
#                     pbar.write(f"Failed to move image '{filename}': {e}")
            
#             pbar.update(1)

#     # 更新图片文件列表，排除已移动的文件
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

#     # Step 2: 筛选白底图
#     print('Step 2: 筛选白底图')
#     move_images_with_white_pixels(grounding_output_path, white_folder, threshold=0.40)

#     # 更新图片文件列表，排除已移动的文件
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

#     # Step 3: 筛选功能卖点图
#     print('Step 3: 筛选功能卖点图')
#     for filename in image_files:
#         img_path = os.path.join(grounding_output_path, filename)
        
#         img = Image.open(img_path)
#         width, height = img.size
        
#         cropped_img = img.crop((0, height // 6, width, height))
#         img_np = np.array(cropped_img)
        
#         result = ocr.ocr(img_np, cls=True)
        
#         if result is None or result[0] is None:
#             continue
            
#         rectangles_with_text = result[0]
#         line_count = len(rectangles_with_text)
        
#         if line_count >= 2:
#             shutil.move(img_path, txt_folder)
#             # print(f"'{filename}' moved to txt folder.")
#         else:
#             # print(f"'{filename}' remains in source folder.")
#             continue

#     # Step 4: 将剩余图片归类到scene
#     print('Step 4: 归类图片到scene')
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]
#     for filename in image_files:
#         img_path = os.path.join(grounding_output_path, filename)
#         shutil.move(img_path, scene_folder)
#         # print(f"Image '{filename}' moved to scene_folder.")

#     # Step 5: 将white中有文本的图片移到txt_folder
#     print('Step 5: 将white中有文本的图片移到txt_folder')
#     image_files = [f for f in os.listdir(white_folder) if f.endswith(('.jpg', '.png'))]
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(white_folder, filename)
#             img = Image.open(img_path)
#             width, height = img.size
#             cropped_img = img.crop((0, height // 6, width, height))
#             img_np = np.array(cropped_img)
#             result = ocr.ocr(img_np, cls=True)
            
#             if result is None or result[0] is None:
#                 pbar.update(1)
#                 continue
                
#             rectangles_with_text = result[0]
#             lines_above_30_count = sum(1 for rectangle in rectangles_with_text if min(math.sqrt((x - rectangle[0][0][0])**2 + (y - rectangle[0][0][1])**2) for x, y in rectangle[0][1:]) > 30)
                
#             if lines_above_30_count >= 2:
#                 shutil.move(img_path, txt_folder)
#                 pbar.write(f"Image '{filename}' moved to txt_folder.")
#             else:
#                 pbar.write(f"Image '{filename}' remains in white folder.")
            
#             pbar.update(1)

# # 主程序
# subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]

# for subfolder in subfolders:
#     print(f"Processing subfolder: {subfolder}")
#     process_subfolder(subfolder)

# print('All processing completed.')

# # 获取当前时间
# current_time = datetime.now()
# print("完成时间:", current_time)



In [1]:
import os
import shutil
from PIL import Image
from paddleocr import PaddleOCR
from tqdm import tqdm
from datetime import datetime
import math
import time
import numpy as np
import pandas as pd

# 定义基础路径
base_path = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'

# 初始化 PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

def move_images_with_white_pixels(source_folder, white_folder, threshold=0.40):
    image_files = [f for f in os.listdir(source_folder) if f.endswith(('.jpg', '.png'))]
    
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(source_folder, filename)
            img = Image.open(img_path)
            img_np = np.array(img)

            # 如果图片中有文本，则不移动到 white_folder
            result = ocr.ocr(img_np, cls=True)
            if result is not None and result[0] is not None:
                ignored_keywords = ['京喜自营', '包邮']
                filtered_data_list = [data for data in result[0] if all(keyword not in data[1][0] for keyword in ignored_keywords)]
                if filtered_data_list:
                    pbar.update(1)
                    continue

            # 计算图片中白色像素的比例
            white_pixels = np.sum(np.all(img_np >= [240, 240, 240], axis=2))
            total_pixels = img_np.shape[0] * img_np.shape[1]
            white_pixels_ratio = white_pixels / total_pixels

            if white_pixels_ratio >= threshold:
                try:
                    shutil.move(img_path, white_folder)
                    pbar.write(f"Image '{filename}' moved to white_folder.")
                except Exception as e:
                    pbar.write(f"Failed to move image '{filename}': {e}")
            
            pbar.update(1)


def process_subfolder(subfolder_path):
    grounding_output_path = os.path.join(subfolder_path, 'grounding_output')
    if not os.path.exists(grounding_output_path):
        # print(f"No grounding_output folder in {subfolder_path}")
        return

    # 创建分类文件夹
    price_folder = os.path.join(grounding_output_path, 'price')
    txt_folder = os.path.join(grounding_output_path, 'txt')
    scene_folder = os.path.join(grounding_output_path, 'scene')
    white_folder = os.path.join(grounding_output_path, 'white')

    for folder in [price_folder, txt_folder, scene_folder, white_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # 获取所有图片文件
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    # Step 1: 筛选价促卖点图
    print('Step 1: 筛选价促卖点图')
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(grounding_output_path, filename)
            
            result = ocr.ocr(img_path, cls=True)
            
            if not result:
                pbar.update(1)
                continue
            
            data_list = result[0]
            
            keywords = ['满', '减', '到手价', '送', '免息', '活动价', '包邮价', '参考价', 
                        r'.*满.*减.*', r'.*满.*-.*', r'.*满.*赠.*',r'.*满.*送.*', r'.*价.*', 
                        '券', '优惠', '用券', '领券', '券', '送', '低至', '立减', '直降', '免息',
                        '¥', '夫', '￥','分钱']
            
            # 忽略“京喜自营”和“包邮”
            ignored_keywords = ['京喜自营', '包邮']
            filtered_data_list = [data for data in data_list if all(keyword not in data[1][0] for keyword in ignored_keywords)]
            
            contains_keyword = any(any(keyword in data[1][0] for keyword in keywords) for data in filtered_data_list)
            
            if contains_keyword:
                try:
                    shutil.move(img_path, price_folder)
                    pbar.write(f"Image '{filename}' moved to price_folder.")
                except Exception as e:
                    pbar.write(f"Failed to move image '{filename}': {e}")
            
            pbar.update(1)

    # 更新图片文件列表，排除已移动的文件
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    # Step 2: 筛选白底图
    print('Step 2: 筛选白底图')
    move_images_with_white_pixels(grounding_output_path, white_folder, threshold=0.40)

    # 更新图片文件列表，排除已移动的文件
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    # Step 3: 筛选功能卖点图
    print('Step 3: 筛选功能卖点图')
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(grounding_output_path, filename)
            
            img = Image.open(img_path)
            width, height = img.size
            
            cropped_img = img.crop((0, height // 6, width, height))
            img_np = np.array(cropped_img)
            
            result = ocr.ocr(img_np, cls=True)
            
            if result is None or result[0] is None:
                pbar.update(1)
                continue
                
            # 忽略“京喜自营”和“包邮”
            ignored_keywords = ['京喜自营', '包邮']
            filtered_data_list = [data for data in result[0] if any(keyword not in data[1][0] for keyword in ignored_keywords)]
            
            rectangles_with_text = filtered_data_list
            line_count = len(rectangles_with_text)
            
            if line_count > 0:
                shutil.move(img_path, txt_folder)
                # print(f"'{filename}' moved to txt folder.")
            else:
                # print(f"'{filename}' remains in source folder.")
                pbar.update(1)

    # Step 4: 将剩余图片归类到scene
    print('Step 4: 归类图片到scene')
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]
    for filename in image_files:
        img_path = os.path.join(grounding_output_path, filename)
        shutil.move(img_path, scene_folder)
        # print(f"Image '{filename}' moved to scene_folder.")

    # Step 5: 将white中有文本的图片移到txt_folder
    print('Step 5: 将white中有文本的图片移到txt_folder')
    image_files = [f for f in os.listdir(white_folder) if f.endswith(('.jpg', '.png'))]
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(white_folder, filename)
            img = Image.open(img_path)
            width, height = img.size
            cropped_img = img.crop((0, height // 6, width, height))
            img_np = np.array(cropped_img)
            result = ocr.ocr(img_np, cls=True)
            
            if result is None or result[0] is None:
                pbar.update(1)
                continue
                
            # 忽略“京喜自营”和“包邮”
            ignored_keywords = ['京喜自营', '包邮']
            filtered_data_list = [data for data in result[0] if all(keyword not in data[1][0] for keyword in ignored_keywords)]
            
            rectangles_with_text = filtered_data_list
            lines_above_30_count = sum(1 for rectangle in rectangles_with_text if min(math.sqrt((x - rectangle[0][0][0])**2 + (y - rectangle[0][0][1])**2) for x, y in rectangle[0][1:]) > 30)
                
            if lines_above_30_count >= 2:
                shutil.move(img_path, txt_folder)
                pbar.write(f"Image '{filename}' moved to txt_folder.")
            else:
                pbar.write(f"Image '{filename}' remains in white folder.")
            
            pbar.update(1)

# 主程序
subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]

for subfolder in subfolders:
    print(f"Processing subfolder: {subfolder}")
    process_subfolder(subfolder)

print('All processing completed.')

# 获取当前时间
current_time = datetime.now()
print("完成时间:", current_time)





Processing subfolder: D://code//data//Lv2期结论//京喜_from_0501//筛选\1047
Step 1: 筛选价促卖点图


Processing images: 100%|██████████| 35/35 [00:07<00:00,  4.91it/s]


Step 2: 筛选白底图


Processing images: 100%|██████████| 35/35 [00:07<00:00,  4.82it/s]


Step 3: 筛选功能卖点图


...

Image '66a30f5eF883fcbc4_e9fcbfe4ad37bf03.jpg' moved to price_folder.


Processing images:  22%|██▏       | 27/125 [00:23<01:48,  1.10s/it]

Image '66ab784eF40a6be83_f21dfc02655503bc.jpg' moved to price_folder.


Processing images:  74%|███████▍  | 93/125 [01:16<00:26,  1.21it/s]


KeyboardInterrupt: 

In [3]:
import os
import shutil
from PIL import Image
from paddleocr import PaddleOCR
from tqdm import tqdm
from datetime import datetime
import math
import time
import numpy as np
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
import os
import shutil
import gc



# 定义基础路径
base_path = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'

# 初始化 PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

def process_subfolder(subfolder_path):
    grounding_output_path = os.path.join(subfolder_path, 'grounding_output')
    if not os.path.exists(grounding_output_path):
        # print(f"No grounding_output folder in {subfolder_path}")
        return

    # 创建分类文件夹
    price_folder = os.path.join(grounding_output_path, 'price')
    txt_folder = os.path.join(grounding_output_path, 'txt')
    scene_folder = os.path.join(grounding_output_path, 'scene')
    white_folder = os.path.join(grounding_output_path, 'white')

    for folder in [price_folder, txt_folder, scene_folder, white_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # 获取所有图片文件
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    # Step 1: 筛选价促卖点图
    print('Step 1: 筛选价促卖点图')
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(grounding_output_path, filename)
            
            result = ocr.ocr(img_path, cls=True)
            
            if not result:
                pbar.update(1)
                continue
            
            data_list = result[0]
            
            keywords = ['满', '减', '到手价', '送', '免息', '活动价', '包邮价', '参考价', 
                        r'.*满.*减.*', r'.*满.*-.*', r'.*满.*赠.*',r'.*满.*送.*', r'.*价.*', 
                        '券', '优惠', '用券', '领券', '券', '送', '低至', '立减', '直降', '免息',
                        '¥', '夫', '￥','分钱']
            
            # 忽略“京喜自营”和“包邮”
            ignored_keywords = ['京喜自营', '包邮']
            filtered_data_list = [data for data in data_list if all(keyword not in data[1][0] for keyword in ignored_keywords)]
            
            contains_keyword = any(any(keyword in data[1][0] for keyword in keywords) for data in filtered_data_list)
            
            if contains_keyword:
                try:
                    shutil.move(img_path, price_folder)
                    pbar.write(f"Image '{filename}' moved to price_folder.")
                except Exception as e:
                    pbar.write(f"Failed to move image '{filename}': {e}")
            
            pbar.update(1)

    # 更新图片文件列表，排除已移动的文件
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    # Step 2: 筛选白底图
    print('Step 2: 筛选白底图')
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(grounding_output_path, filename)
            
            img = Image.open(img_path)
            img_np = np.array(img)

            # 计算图片中白色像素的比例
            white_pixels = np.sum(np.all(img_np >= [240, 240, 240], axis=2))
            total_pixels = img_np.shape[0] * img_np.shape[1]
            white_pixels_ratio = white_pixels / total_pixels

            if white_pixels_ratio >= 0.40:
                try:
                    shutil.move(img_path, white_folder)
                    pbar.write(f"Image '{filename}' moved to white_folder.")
                except Exception as e:
                    pbar.write(f"Failed to move image '{filename}': {e}")
            
            pbar.update(1)

    # 更新图片文件列表，排除已移动的文件
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    # Step 3: 筛选功能卖点图
    print('Step 3: 筛选功能卖点图')
    with tqdm(total=len(image_files), desc="处理图片") as pbar:
        for filename in image_files:
            img_path = os.path.join(grounding_output_path, filename)
            
            # 打开图片
            img = Image.open(img_path)
            width, height = img.size
            
            # 创建一个与原图大小相同的掩码
            mask = Image.new('L', (width, height), 255)
            draw = ImageDraw.Draw(mask)
            
            # 在左下角绘制一个黑色矩形（高200，宽150）
            draw.rectangle([0, height-200, 150, height], fill=0)
            
            # 将掩码应用到图片上
            img_masked = Image.composite(img, Image.new('RGB', img.size, (255, 255, 255)), mask)
            
            # 裁剪图片上部1/6
            cropped_img = img_masked.crop((0, height // 6, width, height))
            
            # 转换为numpy数组
            img_np = np.array(cropped_img)
            
            # 进行OCR识别
            result = ocr.ocr(img_np, cls=True)
            
            if result is None or result[0] is None:
                pbar.update(1)
                continue
            
            # 忽略"京喜自营"和"包邮"
            ignored_keywords = ['京喜自营', '包邮']
            filtered_data_list = [data for data in result[0] if all(keyword not in data[1][0] for keyword in ignored_keywords)]
            
            rectangles_with_text = filtered_data_list
            line_count = len(rectangles_with_text)
            
            if line_count > 0:
                shutil.move(img_path, txt_folder)
                # pbar.write(f"'{filename}' 已移动到 txt 文件夹。")
            else:
                # pbar.write(f"'{filename}' 保留在源文件夹。")
                continue
            
            pbar.update(1)
            gc.collect()


    # Step 4: 将剩余图片归类到scene
    print('Step 4: 归类图片到scene')
    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]
    for filename in image_files:
        img_path = os.path.join(grounding_output_path, filename)
        shutil.move(img_path, scene_folder)
        # print(f"Image '{filename}' moved to scene_folder.")


# 主程序
subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]

for subfolder in subfolders:
    print(f"Processing subfolder: {subfolder}")
    process_subfolder(subfolder)

print('All processing completed.')

# 获取当前时间
current_time = datetime.now()
print("完成时间:", current_time)




Processing subfolder: D://code//data//Lv2期结论//京喜_from_0501//筛选\1047
Step 1: 筛选价促卖点图


Processing images: 100%|██████████| 33/33 [00:06<00:00,  5.04it/s]


Step 2: 筛选白底图


Processing images: 100%|██████████| 33/33 [00:00<00:00, 117.32it/s]


Step 3: 筛选功能卖点图


...

Step 2: 筛选白底图


Processing images: 100%|██████████| 116/116 [00:02<00:00, 57.96it/s]


Image 'txt_66dabc9dFa1a739ad_f11e9c800a6f52f4.jpg' moved to white_folder.
Step 3: 筛选功能卖点图


处理图片: 100%|██████████| 115/115 [01:14<00:00,  1.54it/s]

Step 4: 归类图片到scene
Processing subfolder: D://code//data//Lv2期结论//京喜_from_0501//筛选\饼图数据
All processing completed.
完成时间: 2024-10-18 10:32:57.137667


In [ ]:
# # 这一步是针对京喜场景补充的,因为它里面图片包含"京喜自营"字样,所以要在分类完成之后,再补充一步,把白底图重新清理回去


# import os
# import shutil
# from PIL import Image
# import numpy as np

# def is_mostly_white(image_path, threshold=0.4, white_threshold=240):
#     try:
#         with Image.open(image_path) as img:
#             # 确保图像是RGB模式
#             img = img.convert('RGB')
#             # 转换为numpy数组
#             img_array = np.array(img)
#             # 计算白色像素（RGB值都大于white_threshold的像素视为白色）
#             white_pixels = np.all(img_array > white_threshold, axis=2)
#             white_count = np.sum(white_pixels)
#             total_pixels = img_array.shape[0] * img_array.shape[1]
#             white_ratio = white_count / total_pixels
#             return white_ratio > threshold
#     except Exception as e:
#         print(f"处理图像 {image_path} 时出错: {str(e)}")
#         return False

# def process_folder(root_folder):
#     for dirpath, dirnames, filenames in os.walk(root_folder):
#         if os.path.basename(dirpath) == 'grounding_output':
#             for subfolder in ['txt', 'scene', 'price']:
#                 subfolder_path = os.path.join(dirpath, subfolder)
#                 if os.path.exists(subfolder_path):
#                     white_folder = os.path.join(dirpath, 'white')
#                     if not os.path.exists(white_folder):
#                         os.makedirs(white_folder)
                    
#                     for filename in os.listdir(subfolder_path):
#                         file_path = os.path.join(subfolder_path, filename)
#                         if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
#                             if is_mostly_white(file_path):
#                                 new_path = os.path.join(white_folder, f"{subfolder}_{filename}")
#                                 shutil.move(file_path, new_path)
#                                 print(f"已移动 {file_path} 到 {new_path}")

# # 使用示例
# root_folder = "D://code//data//Lv2期结论//京喜_from_0501//筛选"
# process_folder(root_folder)




# step2 - 对四分类的数据总结

In [ ]:
# import os
# import pandas as pd
# from tqdm import tqdm
# import matplotlib.pyplot as plt

# # 定义基础路径和CSV文件路径
# base_path = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'
# csv_file_path = 'D://code//data//Lv2期结论//京喜_from_0501//京喜数据_from_0501_筛选.csv'

# # 读取CSV文件
# df = pd.read_csv(csv_file_path)

# def extract_matching_part(img_url):
#     if pd.isna(img_url):
#         return None
#     img_url = img_url.split('?')[0]
#     img_url = os.path.splitext(img_url)[0]
#     parts = img_url.split('/')
#     if len(parts) >= 2:
#         return f"{parts[-2]}_{parts[-1]}"
#     return None

# df['matching_part'] = df['img_url'].apply(extract_matching_part)

# def process_grounding_folder(grounding_path):
#     folder_stats = {
#         'price': {'count': 0, 'uv': 0, 'click_uv': 0},
#         'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
#         'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
#         'white': {'count': 0, 'uv': 0, 'click_uv': 0}
#     }

#     for folder_name in ['price', 'txt', 'scene', 'white']:
#         folder_path = os.path.join(grounding_path, folder_name)
#         if os.path.exists(folder_path):
#             for filename in os.listdir(folder_path):
#                 if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
#                     filename_without_ext = os.path.splitext(filename)[0]
#                     filtered_df = df[df['matching_part'] == filename_without_ext]
#                     if not filtered_df.empty:
#                         folder_stats[folder_name]['count'] += 1
#                         folder_stats[folder_name]['uv'] += filtered_df['uv'].sum()
#                         folder_stats[folder_name]['click_uv'] += filtered_df['click_uv'].sum()

#     return folder_stats

# all_stats = {}

# for root, dirs, files in os.walk(base_path):
#     if 'grounding_output' in dirs:
#         grounding_path = os.path.join(root, 'grounding_output')
#         folder_name = os.path.basename(root)
#         stats = process_grounding_folder(grounding_path)
#         all_stats[folder_name] = stats

# # 输出统计结果
# for folder, stats in all_stats.items():
#     print(f"\n统计结果 for {folder}:")
#     for subfolder, data in stats.items():
#         if data['count'] > 0:
#             ctr = data['click_uv'] / data['uv'] if data['uv'] > 0 else 0
#             print(f"  {subfolder}: 图片数量 = {data['count']}, UV = {data['uv']}, Click UV = {data['click_uv']}, CTR = {ctr:.4f}")

# # 绘制每个grounding_output文件夹的饼图
# for folder, stats in all_stats.items():
#     counts = [data['count'] for data in stats.values() if data['count'] > 0]
#     labels = [subfolder for subfolder, data in stats.items() if data['count'] > 0]
    
#     if counts:  # 只有当有数据时才绘图
#         plt.figure(figsize=(12, 9))
#         wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
        
#         # 添加数量标签
#         for i, autotext in enumerate(autotexts):
#             autotext.set_text(f'{autotext.get_text()}\n({counts[i]})')
        
#         plt.title(f"Image Distribution in {folder}")
#         plt.axis('equal')
        
#         # 添加图例
#         plt.legend(wedges, [f"{label} ({count})" for label, count in zip(labels, counts)],
#                    title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        
#         output_chart_path = os.path.join(base_path, f'{folder}_distribution.png')
#         plt.savefig(output_chart_path, bbox_inches='tight')
#         plt.close()
#         print(f"饼图已保存至: {output_chart_path}")

# # 绘制总体分布饼图
# total_counts = [data['count'] for data in total_stats.values() if data['count'] > 0]
# total_labels = [subfolder for subfolder, data in total_stats.items() if data['count'] > 0]

# if total_counts:
#     plt.figure(figsize=(12, 9))
#     wedges, texts, autotexts = plt.pie(total_counts, labels=total_labels, autopct='%1.1f%%', startangle=90)
    
#     # 添加数量标签
#     for i, autotext in enumerate(autotexts):
#         autotext.set_text(f'{autotext.get_text()}\n({total_counts[i]})')
    
#     plt.title("Overall Image Distribution")
#     plt.axis('equal')
    
#     # 添加图例
#     plt.legend(wedges, [f"{label} ({count})" for label, count in zip(total_labels, total_counts)],
#                title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
#     output_chart_path = os.path.join(base_path, 'overall_distribution.png')
#     plt.savefig(output_chart_path, bbox_inches='tight')
#     plt.close()
#     print(f"总体分布饼图已保存至: {output_chart_path}")


    

In [ ]:
# import os
# import pandas as pd
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import openpyxl
# from openpyxl.drawing.image import Image as XLImage

# # 定义基础路径和CSV文件路径
# base_path = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'
# csv_file_path = 'D://code//data//Lv2期结论//京喜_from_0501//京喜数据_from_0501_筛选.csv'

# # 读取CSV文件
# df = pd.read_csv(csv_file_path)

# def extract_matching_part(img_url):
#     if pd.isna(img_url):
#         return None
#     img_url = img_url.split('?')[0]
#     img_url = os.path.splitext(img_url)[0]
#     parts = img_url.split('/')
#     if len(parts) >= 2:
#         return f"{parts[-2]}_{parts[-1]}"
#     return None

# df['matching_part'] = df['img_url'].apply(extract_matching_part)

# def process_grounding_folder(grounding_path):
#     folder_stats = {
#         'price': {'count': 0, 'uv': 0, 'click_uv': 0},
#         'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
#         'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
#         'white': {'count': 0, 'uv': 0, 'click_uv': 0}
#     }

#     for folder_name in ['price', 'txt', 'scene', 'white']:
#         folder_path = os.path.join(grounding_path, folder_name)
#         if os.path.exists(folder_path):
#             for filename in os.listdir(folder_path):
#                 if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
#                     filename_without_ext = os.path.splitext(filename)[0]
#                     filtered_df = df[df['matching_part'] == filename_without_ext]
#                     if not filtered_df.empty:
#                         folder_stats[folder_name]['count'] += 1
#                         folder_stats[folder_name]['uv'] += filtered_df['uv'].sum()
#                         folder_stats[folder_name]['click_uv'] += filtered_df['click_uv'].sum()

#     # 计算每个文件夹的CTR
#     for folder in folder_stats:
#         if folder_stats[folder]['uv'] > 0:
#             folder_stats[folder]['ctr'] = folder_stats[folder]['click_uv'] / folder_stats[folder]['uv']
#         else:
#             folder_stats[folder]['ctr'] = 0

#     return folder_stats

# all_stats = {}

# for root, dirs, files in os.walk(base_path):
#     if 'grounding_output' in dirs:
#         grounding_path = os.path.join(root, 'grounding_output')
#         folder_name = os.path.basename(root)
#         stats = process_grounding_folder(grounding_path)
#         all_stats[folder_name] = stats

# # 创建一个Excel工作簿来保存结果
# workbook = openpyxl.Workbook()
# sheet = workbook.active
# sheet.title = "Image Distribution Stats"

# # 写入表头
# headers = ["Folder", "Subfolder", "Count", "UV", "Click UV", "CTR"]
# for col, header in enumerate(headers, start=1):
#     sheet.cell(row=1, column=col, value=header)

# row = 2  # 从第二行开始写入数据

# # 输出统计结果
# for folder, stats in all_stats.items():
#     print(f"\n统计结果 for {folder}:")
#     for subfolder, data in stats.items():
#         if data['count'] > 0:
#             ctr = data['click_uv'] / data['uv'] if data['uv'] > 0 else 0
#             print(f"  {subfolder}: 图片数量 = {data['count']}, UV = {data['uv']}, Click UV = {data['click_uv']}, CTR = {ctr:.4f}")


# # 定义颜色映射
# color_map = {
#     'price': '#FF6B6B',  # 柔和的红色
#     'txt': '#4ECDC4',    # 青绿色
#     'scene': '#7986CB',  # 淡紫色
#     'white': '#FFD93D'   # 明亮的黄色
# }



# # 绘制每个grounding_output文件夹的饼图并保存数据
# for folder, stats in all_stats.items():
#     counts = [data['count'] for data in stats.values() if data['count'] > 0]
#     labels = [subfolder for subfolder, data in stats.items() if data['count'] > 0]
#     ctrs = [data['ctr'] for data in stats.values() if data['count'] > 0]
#     colors = [color_map.get(label, 'gray') for label in labels]  # 使用颜色映射，如果没有指定则默认为灰色
    
#     if counts:  # 只有当有数据时才绘图
#         plt.figure(figsize=(12, 9))
#         wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
        
#         # 添加数量和CTR标签
#         for i, (autotext, ctr) in enumerate(zip(autotexts, ctrs)):
#             autotext.set_text(f'{autotext.get_text()}\n({counts[i]})\nCTR: {ctr:.4f}')
        
#         plt.title(f"Image Distribution in {folder}")
#         plt.axis('equal')
        
#         # 添加图例
#         plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(labels, counts, ctrs)],
#                    title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        
#         output_chart_path = os.path.join(base_path, f'{folder}_distribution.png')
#         plt.savefig(output_chart_path, bbox_inches='tight')
#         plt.close()
#         print(f"饼图已保存至: {output_chart_path}")

#         # 将图片插入到Excel中
#         img = XLImage(output_chart_path)
#         sheet.add_image(img, f'H{row}')

#         # 写入数据到Excel
#         for subfolder, data in stats.items():
#             if data['count'] > 0:
#                 sheet.cell(row=row, column=1, value=folder)
#                 sheet.cell(row=row, column=2, value=subfolder)
#                 sheet.cell(row=row, column=3, value=data['count'])
#                 sheet.cell(row=row, column=4, value=data['uv'])
#                 sheet.cell(row=row, column=5, value=data['click_uv'])
#                 sheet.cell(row=row, column=6, value=data['ctr'])
#                 row += 1

#         row += 15  # 为下一个文件夹的数据和图表留出空间

# # 绘制总体分布饼图
# total_counts = [data['count'] for data in total_stats.values() if data['count'] > 0]
# total_labels = [subfolder for subfolder, data in total_stats.items() if data['count'] > 0]
# total_ctrs = [data['click_uv'] / data['uv'] if data['uv'] > 0 else 0 for data in total_stats.values() if data['count'] > 0]
# total_colors = [color_map.get(label, 'gray') for label in total_labels]

# if total_counts:
#     plt.figure(figsize=(12, 9))
#     wedges, texts, autotexts = plt.pie(total_counts, labels=total_labels, autopct='%1.1f%%', startangle=90, colors=total_colors)
    
#     # 添加数量和CTR标签
#     for i, (autotext, ctr) in enumerate(zip(autotexts, total_ctrs)):
#         autotext.set_text(f'{autotext.get_text()}\n({total_counts[i]})\nCTR: {ctr:.4f}')
    
#     plt.title("Overall Image Distribution")
#     plt.axis('equal')
    
#     # 添加图例
#     plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(total_labels, total_counts, total_ctrs)],
#                title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
#     output_chart_path = os.path.join(base_path, 'overall_distribution.png')
#     plt.savefig(output_chart_path, bbox_inches='tight')
#     plt.close()
#     print(f"总体分布饼图已保存至: {output_chart_path}")

#     # 将总体分布图片插入到Excel中
#     img = XLImage(output_chart_path)
#     sheet.add_image(img, f'H{row}')

#     # 写入总体数据到Excel
#     sheet.cell(row=row, column=1, value="Overall")
#     for subfolder, data in total_stats.items():
#         if data['count'] > 0:
#             sheet.cell(row=row, column=2, value=subfolder)
#             sheet.cell(row=row, column=3, value=data['count'])
#             sheet.cell(row=row, column=4, value=data['uv'])
#             sheet.cell(row=row, column=5, value=data['click_uv'])
#             sheet.cell(row=row, column=6, value=data['click_uv'] / data['uv'] if data['uv'] > 0 else 0)
#             row += 1

# # 保存Excel文件
# excel_output_path = os.path.join(base_path, 'image_distribution_stats.xlsx')
# workbook.save(excel_output_path)
# print(f"统计结果已保存至Excel文件: {excel_output_path}")


    


统计结果 for 1047:
  txt: 图片数量 = 32, UV = 21486, Click UV = 707, CTR = 0.0329

统计结果 for 12010:
  price: 图片数量 = 3, UV = 568, Click UV = 21, CTR = 0.0370
  txt: 图片数量 = 75, UV = 40107, Click UV = 1540, CTR = 0.0384
  scene: 图片数量 = 3, UV = 870, Click UV = 45, CTR = 0.0517

统计结果 for 12811:
  txt: 图片数量 = 30, UV = 30234, Click UV = 779, CTR = 0.0258
  scene: 图片数量 = 3, UV = 12573, Click UV = 207, CTR = 0.0165

统计结果 for 1349:
  price: 图片数量 = 9, UV = 771, Click UV = 34, CTR = 0.0441
  txt: 图片数量 = 130, UV = 108447, Click UV = 3098, CTR = 0.0286
  scene: 图片数量 = 98, UV = 24358, Click UV = 684, CTR = 0.0281

统计结果 for 1355:
  txt: 图片数量 = 128, UV = 112015, Click UV = 2392, CTR = 0.0214
  scene: 图片数量 = 153, UV = 65909, Click UV = 1497, CTR = 0.0227

统计结果 for 13661:
  price: 图片数量 = 9, UV = 32210, Click UV = 916, CTR = 0.0284
  txt: 图片数量 = 41, UV = 98039, Click UV = 3535, CTR = 0.0361
  scene: 图片数量 = 1, UV = 28, Click UV = 3, CTR = 0.1071

统计结果 for 1476:
  txt: 图片数量 = 16, UV = 51538, Click UV = 1718, CTR = 

NameError: name 'total_stats' is not defined

In [3]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.drawing.image import Image as XLImage

# 定义基础路径和 CSV 文件路径
base_path = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'
csv_file_path = 'D://code//data//Lv2期结论//京喜_from_0501//京喜数据_from_0501_筛选.csv'

# 读取 CSV 文件
df = pd.read_csv(csv_file_path)

def extract_matching_part(img_url):
    if pd.isna(img_url):
        return None
    img_url = img_url.split('?')[0]
    img_url = os.path.splitext(img_url)[0]
    parts = img_url.split('/')
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    return None

df['matching_part'] = df['img_url'].apply(extract_matching_part)

def process_grounding_folder(grounding_path):
    folder_stats = {
        'price': {'count': 0, 'uv': 0, 'click_uv': 0},
        'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
        'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
        'white': {'count': 0, 'uv': 0, 'click_uv': 0}
    }

    for folder_name in ['price', 'txt', 'scene', 'white']:
        folder_path = os.path.join(grounding_path, folder_name)
        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    filename_without_ext = os.path.splitext(filename)[0]
                    filtered_df = df[df['matching_part'] == filename_without_ext]
                    if not filtered_df.empty:
                        folder_stats[folder_name]['count'] += 1
                        folder_stats[folder_name]['uv'] += filtered_df['uv'].sum()
                        folder_stats[folder_name]['click_uv'] += filtered_df['click_uv'].sum()

    # 计算每个文件夹的 CTR
    for folder in folder_stats:
        if folder_stats[folder]['uv'] > 0:
            folder_stats[folder]['ctr'] = folder_stats[folder]['click_uv'] / folder_stats[folder]['uv']
        else:
            folder_stats[folder]['ctr'] = 0

    return folder_stats

all_stats = {}

for root, dirs, files in os.walk(base_path):
    if 'grounding_output' in dirs:
        grounding_path = os.path.join(root, 'grounding_output')
        folder_name = os.path.basename(root)
        stats = process_grounding_folder(grounding_path)
        all_stats[folder_name] = stats

# 创建一个 Excel 工作簿来保存结果
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Image Distribution Stats"

# 写入表头
headers = ["Folder", "Subfolder", "Count", "UV", "Click UV", "CTR"]
for col, header in enumerate(headers, start=1):
    sheet.cell(row=1, column=col, value=header)

row = 2  # 从第二行开始写入数据

# 输出统计结果
for folder, stats in all_stats.items():
    print(f"\n统计结果 for {folder}:")
    for subfolder, data in stats.items():
        if data['count'] > 0:
            ctr = data['click_uv'] / data['uv'] if data['uv'] > 0 else 0
            print(f"  {subfolder}: 图片数量 = {data['count']}, UV = {data['uv']}, Click UV = {data['click_uv']}, CTR = {ctr:.4f}")

# 定义颜色映射
color_map = {
    'price': '#FF6B6B',  # 柔和的红色
    'txt': '#4ECDC4',    # 青绿色
    'scene': '#7986CB',  # 淡紫色
    'white': '#FFD93D'   # 明亮的黄色
}

# 绘制每个 grounding_output 文件夹的饼图并保存数据
for folder, stats in all_stats.items():
    counts = [data['count'] for data in stats.values() if data['count'] > 0]
    labels = [subfolder for subfolder, data in stats.items() if data['count'] > 0]
    ctrs = [data['ctr'] for data in stats.values() if data['count'] > 0]
    colors = [color_map.get(label, 'gray') for label in labels]  # 使用颜色映射，如果没有指定则默认为灰色

    if counts:  # 只有当有数据时才绘图
        plt.figure(figsize=(12, 9))
        wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)

        # 添加数量和 CTR 标签
        for i, (autotext, ctr) in enumerate(zip(autotexts, ctrs)):
            autotext.set_text(f'{autotext.get_text()}\n({counts[i]})\nCTR: {ctr:.4f}')

        plt.title(f"Image Distribution in {folder}")
        plt.axis('equal')

        # 添加图例
        plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(labels, counts, ctrs)],
                   title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

        output_chart_path = os.path.join(base_path, f'{folder}_distribution.png')
        plt.savefig(output_chart_path, bbox_inches='tight')
        plt.close()
        print(f"饼图已保存至: {output_chart_path}")

        # 将图片插入到 Excel 中
        img = XLImage(output_chart_path)
        sheet.add_image(img, f'H{row}')

        # 写入数据到 Excel
        for subfolder, data in stats.items():
            if data['count'] > 0:
                sheet.cell(row=row, column=1, value=folder)
                sheet.cell(row=row, column=2, value=subfolder)
                sheet.cell(row=row, column=3, value=data['count'])
                sheet.cell(row=row, column=4, value=data['uv'])
                sheet.cell(row=row, column=5, value=data['click_uv'])
                sheet.cell(row=row, column=6, value=data['ctr'])
                row += 1

        row += 15  # 为下一个文件夹的数据和图表留出空间

# 计算总体统计数据
total_stats = {
    'price': {'count': 0, 'uv': 0, 'click_uv': 0},
    'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
    'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
    'white': {'count': 0, 'uv': 0, 'click_uv': 0}
}
for stats in all_stats.values():
    for subfolder, data in stats.items():
        total_stats[subfolder]['count'] += data['count']
        total_stats[subfolder]['uv'] += data['uv']
        total_stats[subfolder]['click_uv'] += data['click_uv']

# 计算总体 CTR
for folder in total_stats:
    if total_stats[folder]['uv'] > 0:
        total_stats[folder]['ctr'] = total_stats[folder]['click_uv'] / total_stats[folder]['uv']
    else:
        total_stats[folder]['ctr'] = 0

# 绘制总体分布饼图
total_counts = [data['count'] for data in total_stats.values() if data['count'] > 0]
total_labels = [subfolder for subfolder, data in total_stats.items() if data['count'] > 0]
total_ctrs = [data['click_uv'] / data['uv'] if data['uv'] > 0 else 0 for data in total_stats.values() if data['count'] > 0]
total_colors = [color_map.get(label, 'gray') for label in total_labels]

if total_counts:
    plt.figure(figsize=(12, 9))
    wedges, texts, autotexts = plt.pie(total_counts, labels=total_labels, autopct='%1.1f%%', startangle=90, colors=total_colors)

    # 添加数量和 CTR 标签
    for i, (autotext, ctr) in enumerate(zip(autotexts, total_ctrs)):
        autotext.set_text(f'{autotext.get_text()}\n({total_counts[i]})\nCTR: {ctr:.4f}')

    plt.title("Overall Image Distribution")
    plt.axis('equal')

    # 添加图例
    plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(total_labels, total_counts, total_ctrs)],
               title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

    output_chart_path = os.path.join(base_path, 'overall_distribution.png')
    plt.savefig(output_chart_path, bbox_inches='tight')
    plt.close()
    print(f"总体分布饼图已保存至: {output_chart_path}")

    # 将总体分布图片插入到 Excel 中
    img = XLImage(output_chart_path)
    sheet.add_image(img, f'H{row}')

    # 写入总体数据到 Excel
    sheet.cell(row=row, column=1, value="Overall")
    for subfolder, data in total_stats.items():
        if data['count'] > 0:
            sheet.cell(row=row, column=2, value=subfolder)
            sheet.cell(row=row, column=3, value=data['count'])
            sheet.cell(row=row, column=4, value=data['uv'])
            sheet.cell(row=row, column=5, value=data['click_uv'])
            sheet.cell(row=row, column=6, value=data['click_uv'] / data['uv'] if data['uv'] > 0 else 0)
            row += 1

# 保存 Excel 文件
excel_output_path = os.path.join(base_path, 'image_distribution_stats.xlsx')
workbook.save(excel_output_path)
print(f"统计结果已保存至 Excel 文件: {excel_output_path}")




统计结果 for 1047:
  txt: 图片数量 = 32, UV = 21486, Click UV = 707, CTR = 0.0329

统计结果 for 12010:
  price: 图片数量 = 3, UV = 568, Click UV = 21, CTR = 0.0370
  txt: 图片数量 = 75, UV = 40107, Click UV = 1540, CTR = 0.0384
  scene: 图片数量 = 3, UV = 870, Click UV = 45, CTR = 0.0517

统计结果 for 12811:
  txt: 图片数量 = 30, UV = 30234, Click UV = 779, CTR = 0.0258
  scene: 图片数量 = 3, UV = 12573, Click UV = 207, CTR = 0.0165

统计结果 for 1349:
  price: 图片数量 = 9, UV = 771, Click UV = 34, CTR = 0.0441
  txt: 图片数量 = 130, UV = 108447, Click UV = 3098, CTR = 0.0286
  scene: 图片数量 = 98, UV = 24358, Click UV = 684, CTR = 0.0281

统计结果 for 1355:
  txt: 图片数量 = 132, UV = 114708, Click UV = 2453, CTR = 0.0214
  scene: 图片数量 = 153, UV = 65909, Click UV = 1497, CTR = 0.0227

统计结果 for 13661:
  price: 图片数量 = 9, UV = 32210, Click UV = 916, CTR = 0.0284
  txt: 图片数量 = 41, UV = 98039, Click UV = 3535, CTR = 0.0361
  scene: 图片数量 = 1, UV = 28, Click UV = 3, CTR = 0.1071

统计结果 for 1476:
  txt: 图片数量 = 16, UV = 51538, Click UV = 1718, CTR = 

In [20]:
import os
import pandas as pd

# 存储图片信息的列表
image_info = []

# 一级文件夹路径
root_folder = 'D://code//data//Lv2期结论//京喜_from_0501//筛选'

# 遍历一级文件夹下的所有二级文件夹
for sub_folder in os.listdir(root_folder):
    if sub_folder.isdigit():  # 只处理数字命名的二级文件夹
        grounding_folder = os.path.join(root_folder, sub_folder, 'grounding_output')
        if os.path.exists(grounding_folder):
            for sub_sub_folder in ['price', 'txt', 'white', 'scene']:
                sub_sub_folder_path = os.path.join(grounding_folder, sub_sub_folder)
                if os.path.exists(sub_sub_folder_path):
                    for image_file in os.listdir(sub_sub_folder_path):
                        if "txt_" in image_file:
                            image_file = image_file.replace("txt_", "")
                        elif "price_" in image_file:
                            image_file = image_file.replace("price_", "")
                        image_info.append([image_file, sub_sub_folder, sub_folder])

# 创建DataFrame并保存为Excel
df = pd.DataFrame(image_info, columns=['图片名', '分类', 'cid3'])
df.to_excel('D://code//data//Lv2期结论//京喜_from_0501//筛选//分类数据image_info.xlsx', index=False)




In [19]:
import pandas as pd

# 读取Excel文件
data = pd.read_csv('D://code//data//Lv2期结论//京喜_from_0501//京喜数据_from_0501_筛选.csv')

# 按照img_url列进行聚合，并对指定列进行求和
aggregated_data = data.groupby('img_url').agg({
    'uv':'sum',
    'click_uv':'sum',
    'gmv_cj':'sum',
    'sale_qtty_cj':'sum'
}).reset_index()

# 将原始数据中与聚合相关的列合并到聚合后的数据中
aggregated_data = pd.merge(aggregated_data, data[['sku', 'img_url','img_type', 'bu_id', 'cid1', 'cid2', 'cid3', 'main_brand_code','shop_id']], on='img_url', how='left')

# 保存为新的Excel文件
aggregated_data.to_excel('D://code//data//Lv2期结论//京喜_from_0501//url加总原始数据.xlsx', index=False)




In [22]:
csv_file_path = 'D://code//data//Lv2期结论//京喜_from_0501//url加总原始数据.xlsx'


# 读取 CSV 文件
df = pd.read_excel(csv_file_path)

def extract_matching_part(img_url):
    if pd.isna(img_url):
        return None
    img_url = img_url.split('?')[0]
    img_url = os.path.splitext(img_url)[0]
    parts = img_url.split('/')
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    return None

df['matching_part'] = df['img_url'].apply(extract_matching_part)



In [23]:
df.head()



,img_url,uv,click_uv,gmv_cj,sale_qtty_cj,sku,img_type,bu_id,cid1,cid2,cid3,main_brand_code,shop_id,matching_part
0,jfs/t1/100004/33/49060/76171/66c2bcffF8ae95d14...,161,2,0.00,0,10096570673481,0,10995,670,686,1047,14151,13496968.0,66c2bcffF8ae95d14_7c651dcb5832f305
1,jfs/t1/100024/34/50479/106232/66dab790Fda90ad8...,79,4,9.18,2,10103620326168,0,10995,16750,16752,16777,14151,13496968.0,66dab790Fda90ad89_8ab1fdfecb747a9b
2,jfs/t1/100026/5/52925/100421/66fc0e24F88cb6f8a...,391,12,0.00,0,10107750989796,0,10995,6196,6219,6221,14151,13496968.0,66fc0e24F88cb6f8a_45a02d05d6f30c21
3,jfs/t1/100026/5/52925/100421/66fc0e24F88cb6f8a...,391,12,0.00,0,10107750989796,0,10995,6196,6219,6221,14151,13496968.0,66fc0e24F88cb6f8a_45a02d05d6f30c21
4,jfs/t1/100033/26/51602/167543/66f19a6cF3dcd323...,791,44,219.80,38,10106909851270,0,10995,15901,15902,15908,14151,13496968.0,66f19a6cF3dcd3236_6cbd5fa9693c541c


In [40]:
import pandas as pd

df_x = pd.read_excel('D://code//data//Lv2期结论//京喜_from_0501//筛选//分类数据image_info.xlsx')

# 确保列数据类型匹配（如果需要）
df['matching_part'] = df['matching_part'].astype(str)
df_x['图片名'] = df_x['图片名'].apply(lambda x: str(x).replace('.jpg', '') if isinstance(x, (str, bytes)) else x)

# 根据条件进行拼接
merged_df = pd.merge(df, df_x, left_on='matching_part', right_on='图片名', how='right')



In [44]:
merged_df.to_excel('D://code//data//Lv2期结论//京喜_from_0501//sku分类数据表.xlsx', index=False)



In [46]:
merged_df.head()

,img_url,uv,click_uv,gmv_cj,sale_qtty_cj,sku,img_type,bu_id,cid1,cid2,cid3_x,main_brand_code,shop_id,matching_part,图片名,分类,cid3_y
0,jfs/t1/193817/9/42691/119291/667e8c77Fea571c39...,1229.0,32.0,50.96,5.0,1.010354e+13,0.0,10995.0,670.0,686.0,1047.0,14151.0,13496968.0,667e8c77Fea571c39_828c9e4956e6f12f,667e8c77Fea571c39_828c9e4956e6f12f,txt,1047
1,jfs/t1/193817/9/42691/119291/667e8c77Fea571c39...,1229.0,32.0,50.96,5.0,1.010354e+13,0.0,10995.0,670.0,686.0,1047.0,14151.0,13496968.0,667e8c77Fea571c39_828c9e4956e6f12f,667e8c77Fea571c39_828c9e4956e6f12f,txt,1047
2,jfs/t1/193817/9/42691/119291/667e8c77Fea571c39...,1229.0,32.0,50.96,5.0,1.010354e+13,0.0,10995.0,670.0,686.0,1047.0,14151.0,13496968.0,667e8c77Fea571c39_828c9e4956e6f12f,667e8c77Fea571c39_828c9e4956e6f12f,txt,1047
3,jfs/t1/235708/21/21076/50933/667e8d8cFd1e12884...,961.0,13.0,8.80,2.0,1.010541e+13,0.0,10995.0,670.0,686.0,1047.0,14151.0,13496968.0,667e8d8cFd1e12884_b77cc342352a7b11,667e8d8cFd1e12884_b77cc342352a7b11,txt,1047
4,jfs/t1/235708/21/21076/50933/667e8d8cFd1e12884...,961.0,13.0,8.80,2.0,1.010541e+13,0.0,10995.0,670.0,686.0,1047.0,14151.0,13496968.0,667e8d8cFd1e12884_b77cc342352a7b11,667e8d8cFd1e12884_b77cc342352a7b11,txt,1047
